In [3]:
from pysis.flowsheet import Simulation

import numpy
from scipy.stats import _qmc
import pysis.flowsheet as sim_f






In [4]:
fs = Simulation(path=r"C:\Users\LuanNogueira\Downloads\Ic-Metamodelagem\DWSIMSIMS\HidrogenaçãoCO2A")
# fs.update_flowsheet()
# fs.set_visible(visibility=1)
# operations=fs.Operations
# materialStreams=fs.MatStreams
# distillationColumn=sim_f.DistillationColumn(fs.Operations['T-100'].COMObject)
# Pfr=sim_f.PFR(fs.Operations['PFR-100'].COMObject)
# fs.Solver.CanSolve=True

In [ ]:
def DesignVariables():
    return{
    'VolumeReator':Pfr.get_volume(),

    "AreaTC":[fs.Operations[u].COMObject.UAValue*1000/1700 for u in operations.keys() if operations[u].classification=='Heat Transfer Equipment' and any([x=='HeatTransferArea' for x in fs.Operations[u].COMObject.__dir__()])],

    "VazaoWasteWater":fs.MatStreams['LIQ'].COMObject.IdealLiquidVolumeFlowValue*1000,

    "PotenciaFornalha":fs.EnerStreams['CustoAquecimento'].get_power()/1e3,

    "VolumePackingTD":fs.Operations['CustoDestilação'].COMObject.Cell('A1').CellValue,

    "PotenciaCompressores":[fs.Operations[u].COMObject.EnergyValue for u in operations.keys() if operations[u].classification=='Rotating Equipment'],

    "MassaProcessVessel":[fs.Operations['ProcessVessel'].COMObject.Cell('B7').CellValue, fs.Operations['ProcessVessel'].COMObject.Cell('B8').CellValue]
    }
def listDesignVariables(y:dict):
    x=[]

    for u in list(y.values()):
        if type(u)==list:
            for _ in u:
                x.append(_)
        else:
            x.append(u)
    return x

def UCEquipamentos():

    design_variables=DesignVariables()
    return{
    'CustoReator':4.00*(61500+32500*(design_variables['VolumeReator'])**0.80),

    'CustoTrocadoresDeCalor':sum([1.0*(28000+54*u**1.20) for u in design_variables['AreaTC']]),

    'CustoTorreResfriamento':2.5*(170000+1500*design_variables['VazaoWasteWater']**0.90),

    'CustoFornalha':2.5*(80000+10900*(design_variables['PotenciaFornalha'])**0.80),

    'CustoDestilação':4000*design_variables['VolumePackingTD'],

    'CustoComp':sum([2.5*(580000+20000*(u)**0.6) for u in design_variables['PotenciaCompressores']]),

    'CustoProcessVessel':sum([4.0*(17400+79*(u)**0.85) for u in design_variables['MassaProcessVessel']])
}
def CustoEquipamentos():
    return sum(list(UCEquipamentos().values()))

def QntdUtilidades():
    return {
    'VazaoUtilidadeFria':fs.MatStreams['UtilidadeFria'].COMObject.IdealLiquidVolumeFlowValue*3600,

    'PotenciaAquecimento':fs.EnerStreams['CustoAquecimento'].get_power()/1e6*3600,

    'VazaoWasteWater':fs.MatStreams['LIQ'].COMObject.IdealLiquidVolumeFLowValue*3600,
    
    'PotenciaEletricidade':fs.EnerStreams['CustoEletricidade'].get_power()/1e3*360
}

def CustoUtilidades():

    qntdutilidades=QntdUtilidades()
    return{
    'CustoResfriamento':qntdutilidades['VazaoUtilidadeFria']*0.03,

    'CustoAquecimento':qntdutilidades['PotenciaAquecimento']*12.1,

    'CustoWasteWater':qntdutilidades['VazaoWasteWater']*1.5,

    'CustoEletricidade':qntdutilidades['PotenciaEletricidade']*94.5
}

def CustoTotalUtilidades():
    return sum(list(CustoUtilidades().values()))

def VazaoMP():
    return {
    'QntdH2':fs.MatStreams['9'].COMObject.IdealLiquidVolumeFlowValue*3600,
    # ton
    'QntdCO2':fs.MatStreams['1'].get_massflow()/1000
}
def CustoMP():
    return{
    'CustoH2':VazaoMP()['QntdH2']*0.165,
    'CustoCO2':VazaoMP()['QntdCO2']*95.50
}

def CustoTotalMP():
    return sum(list(CustoMP().values()))



In [ ]:
def simulated_values() ->list: 
    
   return (listDesignVariables(DesignVariables())+list(QntdUtilidades().values())+list(VazaoMP().values()))

def obj_f():

    return [CustoEquipamentos(),CustoTotalMP(),CustoTotalUtilidades()]

def set_dof(x:list):
    
    fs.Solver.CanSolve=False

    materialStreams['13'].set_temperature(x[0],'C')

    materialStreams['9'].set_molarflow(x[1],'kgmole/h')
    
    fs.Operations['TEE-100'].SplitsValue=(x[2],1-x[2])

    distillationColumn.set_specifications(spec='Reflux Ratio', value=x[3])
    
    Pfr.set_volume(x[4])

    materialStreams['10'].set_pressure(x[5],'kpa')
    materialStreams['8'].set_pressure(x[5],'kpa')
    materialStreams['20'].set_pressure(x[5],'kpa')
    
    materialStreams['REC-2'].set_pressure(x[5],'kpa')
    fs.Solver.CanSolve = True

def RReciclo():
    materialStreams['REC'].set_molarflow(5.204e4)
    materialStreams['REC'].set_compmolarflow(materialStreams['REC-2'].get_compmolarflow())
streams_pressure=dict(zip(list(fs.MatStreams.keys()),[fs.MatStreams[u].get_pressure() for u in list(fs.MatStreams.keys())]))

def reset_stream():
    unconverged_streams=[i for i in list(fs.MatStreams.keys()) if fs.MatStreams[i].get_massflow()<0]
    streams_pressure=dict(zip(list(fs.MatStreams.keys()),[fs.MatStreams[u].get_pressure() for u in list(fs.MatStreams.keys())]))
    for i in unconverged_streams:
        if fs.MatStreams[i].COMObject.CanModifyStream == True:
            fs.MatStreams[i].set_pressure(100)
            fs.MatStreams[i].set_pressure(streams_pressure[i])
def run(x:list):
    
    y=[]
    for _ in x:
       
        set_dof(_)

        trials=0
        is_converged=False
        is_valid=False
        while(is_converged==False or is_valid==False or stream_convergeance==False) and trials<=3:
            set_dof(_)
            if trials>0:
                RReciclo()
            try:
                
                sv=simulated_values()
                is_valid=all([i>-3000 for i in sv])
            except:
                print ('erro')
                is_valid=False
            flag_streams=dict(list(zip(list(fs.MatStreams.keys()),[i>=0 for i in [f.get_massflow() for f in fs.MatStreams.values()]]))+list(zip(list(fs.EnerStreams.keys()),[i>=0 for i in [f.get_power() for f in fs.EnerStreams.values()]])))
            stream_convergeance=all([i>=0 for i in [f.get_massflow() for f in fs.MatStreams.values()]+[f.get_power() for f in fs.EnerStreams.values()]])
            is_converged=distillationColumn.get_convergence()
            if stream_convergeance==False:
                for i,j in flag_streams.items():
                    if (j==False):
                        print('Corrente zikada:',i)
            trials+=1
        print(DesignVariables())
        reset_stream()
        y.append(obj_f()+[is_valid, is_converged, stream_convergeance])
    
    
    return y
    # fs.close()

def LHS(n):
    sampler=_qmc.LatinHypercube(d=6)
    sample=sampler.random(n)
    _qmc.discrepancy(sample)
    l_bounds=[180,4500,0.001,1.25,35,4500]
    u_bounds=[240,6500,0.05,1.8,55,5500]
    x=list(_qmc.scale(sample,l_bounds, u_bounds))
    return x


# run(LHS(1))

NameError: name 'fs' is not defined

In [ ]:
# reset_stream()

In [ ]:
# - Cálculo da área
#  - HYSYS falha e volta a funcionar?
#  - BKP enquanto roda os experimentos